In [9]:
import pandas as pd
import math
import numpy as np
from ast import literal_eval
from bisect import bisect_left
from src.usa.plot_utils import plot_new_facilities, plot_state
from src.usa.utils import compute_medical_deserts
from src.facility_location.utils import compute_minimum_distances
from src.usa.constants import state_names
from src.usa.facilities import PharmaciesTop3, ChildCare, FDICInsuredBanks, Hospitals, NursingHomes, PrivateSchools, UrgentCare
from haversine import haversine
from src.usa.states import USAState
import plotly.graph_objects as go
import os
import csv
from tqdm import tqdm
import sys


In [2]:
os.chdir('..')

In [6]:
ALL_FACILITY_TYPES = [PharmaciesTop3, ChildCare, FDICInsuredBanks, Hospitals, NursingHomes, PrivateSchools, UrgentCare]
facility_names = [facility.name for facility in ALL_FACILITY_TYPES]

In [7]:
def get_facility_weights(solutions):
    weights = dict()
    thresholds = [5, 10, 25, 50, 100, np.inf]
    
    for facility in set(solutions[1] + solutions[2] + solutions[3]):
        weights[facility] = 0
        for i in range(1, 4):
            if facility in solutions[i]:
                id = solutions[i].index(facility)
                threshold = bisect_left(a=thresholds, x=id)
                weights[facility] += math.pow(1/thresholds[threshold], 1)

    return weights


In [8]:
def compute_distances_to_facility(facility_longitude, facility_latitude, longitudes, latitudes):
    distances = []
    for i in range(len(longitudes)):
        distances.append(haversine(point1=(facility_latitude, facility_longitude), point2=(latitudes[i], longitudes[i])))
    
    return distances


def facility_gain(distances_to_existing_facilities, distances_to_facility, urban, weight):
    reduction = 0
    for i in range(len(distances_to_existing_facilities)):
        facility_gain = distances_to_existing_facilities[i] - distances_to_facility[i]
        facility_gain = max(facility_gain, 0)
        if urban[i]:
            facility_gain = facility_gain * 5
        reduction = reduction + facility_gain
    
    return reduction * weight


def find_facility_with_highest_score(scores):
    facility = -1
    score = 0
    for f in scores.keys():
        if scores[f] >= score:
            facility = f
            score = scores[f]
            
    return facility


def compute_new_minimum_distances(distances_to_existing_facilities, distances_to_facility):
    new_distances = []
    for i in range(len(distances_to_existing_facilities)):
        new_distances.append(min(distances_to_existing_facilities[i], distances_to_facility[i]))
    
    return new_distances


def greedy_algorithm_to_combine_facilities(solutions, census_df, facility_name, K=25):
    list_of_new_facilities = set(solutions[1] + solutions[2] + solutions[3])
    
    facilities_output_order = []
    existing_distances = list(census_df['closest_distance_' + facility_name])
    longitudes = list(census_df['Longitude'])
    latitudes = list(census_df['Latitude'])
    urban = list(census_df['urban'])
    
    distances_to_new_facilities = {
        facility: compute_distances_to_facility(census_df.loc[facility]['Longitude'], census_df.loc[facility]['Latitude'], longitudes, latitudes) for facility in list_of_new_facilities
    }
    weights = get_facility_weights(solutions)
    
    while len(facilities_output_order) < K:
        scores = dict()
        for facility in list_of_new_facilities:
            scores[facility] = facility_gain(existing_distances, distances_to_new_facilities[facility], urban, weights[facility])
    
        f = find_facility_with_highest_score(scores)
        facilities_output_order.append(f)
        list_of_new_facilities.remove(f)

        existing_distances = compute_new_minimum_distances(existing_distances, distances_to_new_facilities[f])
        
    return facilities_output_order


In [ ]:
for facility_name in facility_names:
    print('Facility:', facility_name, file=sys.stderr)
    
    filename = 'data/usa/new_facilities/' + facility_name + '/new_facilities_combined.csv'
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['State', 5, 10, 25, 50, 100])

    for state_name in tqdm(state_names):
        State = USAState(state_name)
        census_df = State.get_census_data(level='blockgroup')
    
        df1 = pd.read_csv('data/usa/new_facilities/' + facility_name + '/new_facilities_1.csv', index_col=0)
        df2 = pd.read_csv('data/usa/new_facilities/' + facility_name + '/new_facilities_2.csv', index_col=0)
        df3 = pd.read_csv('data/usa/new_facilities/' + facility_name + '/new_facilities_inf.csv', index_col=0)
    
        solutions = {
            1: literal_eval(df1.loc[state_name]['100']),
            2: literal_eval(df2.loc[state_name]['100']),
            3: literal_eval(df3.loc[state_name]['100'])
        }
        
        new_facilities = greedy_algorithm_to_combine_facilities(solutions, census_df, facility_name, K=100)

        with open(filename, 'a') as file:
            writer = csv.writer(file)
            writer.writerow([state_name, new_facilities[:5],
                             new_facilities[:10],
                             new_facilities[:25],
                             new_facilities[:50],
                             new_facilities[:100]])


Facility: top_3_pharmacy_chains
 74%|███████▍  | 37/50 [06:56<02:26, 11.24s/it]